In [11]:
# !pip3 install chardet -i https://mirrors.aliyun.com/pypi/simple/

In [3]:
import glob
import os
from tqdm import tqdm
import numpy as np

import IPython
def play_audio(data, rate):
    IPython.display.display(IPython.display.Audio(data=data,rate=rate))

In [ ]:
hindilabels = [
    "CMU-ARCTIC/SLT",
    "L2-ARCTIC/ASI",
    "L2-ARCTIC/TNI",
    "PD-AST/SLT/English",
    "PD-AST/SLT/Hindi",
    "PD-AST/ASI/Hindi",
    "PD-AST/TNI/Hindi",
    # "VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating_small/100000",
    # "VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating/100000",
    # "VTN_fine-tuning_nocondition_mix2synVCTK3hr_melmel_hubert_norepeating_small/100000",
    # "VTN_fine-tuning_nocondition_mix2synVCTK3hr_melmel_hubert_norepeating/100000",
    "VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating_small/100000",
    "VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating/100000",
    "VTN_fine-tuning_nocondition_mix2synVCTK3hr_80mel80mel_hubert_norepeating_small/100000",
    "VTN_fine-tuning_nocondition_mix2synVCTK3hr_80mel80mel_hubert_norepeating/100000",
]

koreanlabels = [
    "L2-ARCTIC/HKK",
    "L2-ARCTIC/YDCK",
    "PD-AST/SLT/English",
    "PD-AST/SLT/Korean",
    "PD-AST/HKK/Korean",
    "PD-AST/YDCK/Korean",
]

In [115]:
sr = 16000
result_base_dir = "./audio/20240815_spaccent/"

experiments = {
    "hindi": [
        "CMU-ARCTIC/SLT",
        "PD-AST/SLT/English",
        "L2-ARCTIC/ASI",
        "L2-ARCTIC/TNI",
        "PD-AST/SLT/Hindi",
        "PD-AST/ASI/Hindi",
        "PD-AST/TNI/Hindi",
        "VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating_small/100000",
        "VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating/100000",
        "VTN_fine-tuning_nocondition_mix2synVCTK3hr_melmel_hubert_norepeating_small/100000",
        "VTN_fine-tuning_nocondition_mix2synVCTK3hr_melmel_hubert_norepeating/100000",
        "VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating_small/100000",
        "VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating/100000",
        "VTN_fine-tuning_nocondition_mix2synVCTK3hr_80mel80mel_hubert_norepeating_small/100000",
        "VTN_fine-tuning_nocondition_mix2synVCTK3hr_80mel80mel_hubert_norepeating/100000",
    ],
    "korean": [
        "CMU-ARCTIC/SLT",
        "PD-AST/SLT/English",
        "L2-ARCTIC/HKK",
        "L2-ARCTIC/YDCK",
        "PD-AST/SLT/Korean",
        "PD-AST/HKK/Korean",
        "PD-AST/YDCK/Korean",
    ],
}

import chardet
def openfile(txtfile):
    with open(txtfile, "rb") as f:
        raw_data = f.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']

    with open(txtfile, "r", encoding=encoding) as f:
        content = f.readlines()
    return content
files = glob.glob("../../seq2seq-vc/datasetgeneration/LLM_responses/08-A_multi-lingual_text/*.npy")
files.sort()
groundtruth_dir = {}
for file in files:
    key = os.path.basename(file)[:-4]
    text = np.load(file, allow_pickle=True).item()["Original English"]
    groundtruth_dir[key] = text

# MUSHRA (Hindi)

In [116]:
exid = "hindi"
testmode = "accentedness" # naturalness or accentedness
num = 10
start = 0

testname = "Speech Naturalness Evaluation" if testmode=="naturalness" else "Speech Accentedness Evaluation"
def init():
    whole_text = f"""var TestConfig = {{
  "TestName": "{testname}",
  "RateScalePng": "img/scale_abs.png",
  "RateScaleBgPng": "img/scale_abs_background.png",
  "RateMinValue": 0,
  "RateMaxValue": 100,
  "RateDefaultValue":50,
  "ShowFileIDs": true,
  "ShowResults": false,
  "LoopByDefault": false,
  "EnableABLoop": true,
  "EnableOnlineSubmission": true,
  "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
  "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
  "RandomizeTestOrder": true,
  "MaxTestsPerRun": 10,
  "RequireMaxRating": false,
  "AudioRoot": "",
  "Testsets":
["""
    return whole_text

whole_text = init()
dirnames = experiments[exid]
dirname = dirnames[0]
dirname = f'{"___".join(dirname.split("/"))}'
filenames = [os.path.basename(a)[:-4] for a in glob.glob(result_base_dir+f"{dirname}/*.wav")]
filenames.sort()
for b, basename in enumerate(filenames[start:start+num]):
    spk, name = basename.split("_")
    gt_path = result_base_dir + f"ground_truth/{basename}.wav"
    text = groundtruth_dir[basename]
    if testmode=="accentedness":
        text = f"Accent: {exid[0].upper()+exid[1:]} --- {text}"
    starter = f"""
  {{
    "Name": "{basename}",
    "TestID": "NATHindi---{basename}",
    "Text": "{text}",
    "Files": {{\n"""
    modelpaths = """"""
    for dirname in dirnames:
        dirname = f'{"___".join(dirname.split("/"))}'
        modelpath = result_base_dir+f"{dirname}/{basename}.wav"
        added = f'      "{dirname}": "{modelpath}",\n'
        modelpaths += added
    ##### One more sample #####
    np.random.seed(b)
    dirname = dirnames[np.random.randint(len(dirnames))]
    dirname = f'{"___".join(dirname.split("/"))}'
    modelpath = result_base_dir+f"{dirname}/{basename}.wav"
    added = f'      "{dirname}---2": "{modelpath}",\n'
    modelpaths += added
    ##########################
    modelpaths += f'    }}\n'
    modelpaths += f'  }},'
    whole_text += starter+modelpaths
whole_text += "\n]\n}}"
print(whole_text[:-1])

var TestConfig = {
  "TestName": "Speech Accentedness Evaluation",
  "RateScalePng": "img/scale_abs.png",
  "RateScaleBgPng": "img/scale_abs_background.png",
  "RateMinValue": 0,
  "RateMaxValue": 100,
  "RateDefaultValue":50,
  "ShowFileIDs": true,
  "ShowResults": false,
  "LoopByDefault": false,
  "EnableABLoop": true,
  "EnableOnlineSubmission": true,
  "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
  "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
  "RandomizeTestOrder": true,
  "MaxTestsPerRun": 10,
  "RequireMaxRating": false,
  "AudioRoot": "",
  "Testsets":
[
  {
    "Name": "arctic_a0024",
    "TestID": "NATHindi---arctic_a0024",
    "Text": "Accent: Hindi --- It was my reports from the north which chiefly induced people to buy.",
    "Files": {
      "CMU-ARCTIC___SLT": "./audio/20240815_spaccent/CMU-ARCTIC___SLT/arctic_a0024.wav",
      "PD-AST___SLT___English": "./audio/20240815_spaccent/PD-AST___SLT___English/arctic_a0024.wav",
      "L2-ARCTIC___ASI": "./audi